# Import the needed packages 

In [ ]:
import erroranalysis as err
import auxfunctions as aux
import matplotlib.pyplot as plt

In [ ]:
Z, C, W, M = aux.load_data('./data/SG_data.csv')
ZRef, CRef, WRef, MRef = aux.load_data('./data/SG_data_ref.csv')
box = [-3.66, -1.75, 0, 3.66, 1.75, 0.45]

In [ ]:
RMSv = err.Root_Mean_Squared_Velocity(ZRef, CRef, WRef, 'Zonal')

# Create a line plot
plt.plot(RMSv)

# Add labels and title (optional)
plt.xlabel('Time Step')
plt.ylabel('RMSv')
plt.title('RMSv')

# Display the plot (if running in a Jupyter Notebook, this step might not be necessary)
plt.show()

In [ ]:
euclid_error = err.Weighted_Euclidian_Error(Z, ZRef, MRef, 5, 3, box)

print(euclid_error)

In [ ]:
sinkhorn_loss = err.Sqrt_Sinkhorn_Loss(Z, M, ZRef, MRef, 0.001, 5, 2.5, box)

print(sinkhorn_loss)